# Purpose
This notebook describes the typical activities carried out at the beginning to a project / thread when customer shares new data. We will be trying to understand the tables, columns and information flow. Typically we also look for data issues and confirm with respective owners for resolution. At the end of this activity, the data sources and their treatment is finalized. Code in this notebook will not be part of the production code.

This data can be downloaded from [here](https://drive.google.com/file/d/14FaUvJuhQLZevvqQcnOZZxvkvKbdPhZ7/view?usp=sharing)

# Initialization

In [6]:
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import os
import os.path as op
import pandas as pd 
import great_expectations as ge
from dateutil.relativedelta import relativedelta
os.environ['TA_DEBUG'] = "False"
os.environ['TA_ALLOW_EXCEPTIONS'] = "True"

In [8]:
import warnings

warnings.filterwarnings('ignore', message="The sklearn.metrics.classification module", category=FutureWarning)
warnings.filterwarnings('ignore', message=".*title_format is deprecated. Please use title instead.*")

In [31]:
%%time
from ta_lib.core.api import (
    create_context,
    get_package_path,
    display_as_tabs,
    initialize_environment,
    string_cleaning,
    setanalyse,
    merge_expectations
)
import ta_lib.core.api as dataset
import ta_lib.eda.api as analysis
import ta_lib.reports.api as health

Wall time: 0 ns


In [10]:
# Initialization
initialize_environment(debug=False, hide_warnings=True)

# Data

## Background

Customer is a distributor of consumption goods. There are a variety of data files which we would be using. For more refer to the readme in the data archive. Business wants to know whether a particular customer will churn or not with Machine learning. 

Below is an overview of each of the keys in the dataset:

| | Key | Dataset | Details |
|--|--|--|--|
| 1 | opportunity_id | opportunity.xlsb | The monthly primary sales from the manufacturer to each distributor. The column pri_sales_amount indicates the sales. We will use this column for defining churn of a distributor, i.e. whenever the sales turn 0 or negative, the distributor would be defined as churned. The cutoff or reference date we'll use to predict future churn is 2013-05-01 |
| 2 | product_id | product.xlsb | The monthly secondary business, with column sec_netvalues identifying the total sales from each distributor |

In [11]:
%%time
!pip install pyxlsb
opp_xlsb = pd.read_excel('../../data/raw/opportunity/opportunity.xlsb', engine='pyxlsb')
pro_xlsb = pd.read_excel('../../data/raw/opportunity/product.xlsb', engine='pyxlsb')
opp_xlsb.shape, pro_xlsb.shape

  Using cached pyxlsb-1.0.9-py2.py3-none-any.whl (23 kB)
Wall time: 1min 7s


((313571, 16), (142431, 8))

In [12]:
opp_xlsb.to_csv('../../data/raw/opportunity/csv/opportunity.csv', index=False)
pro_xlsb.to_csv('../../data/raw/opportunity/csv/product.csv', index=False)

In [13]:
config_path = op.join('conf', 'config.yml')
context = create_context(config_path)
dataset.list_datasets(context)

['/raw/product',
 '/raw/opportunity',
 '/cleaned/target',
 '/cleaned/product',
 '/cleaned/opportunity',
 '/processed/merged',
 '/train/merged/features',
 '/train/merged/target',
 '/test/merged/features',
 '/test/merged/target',
 '/score/merged/output']

In [14]:
# Loading all datasets in a loop
data = dict()
for i in dataset.list_datasets(context):
    if '/raw/' in i:
        key_ = i.replace('/raw/','')+'_df'
        data[key_] = dataset.load_dataset(context,i)
        # Standardize column names
        data[key_].columns = string_cleaning(data[key_].columns,lower=True)

## Exploratory Data Analysis

### Shape of Data

In [15]:
(
    pd.DataFrame({x:data[x].shape for x in data.keys()})
    .T
    .rename(columns={0:'rows',1:'columns'})
    .sort_values('rows',ascending=False)
)

,rows,columns
opportunity_df,313571,16
product_df,142431,8


## Variable summary

In [16]:
summaries = [analysis.get_variable_summary(data[x]).assign(Null_Values=pd.Series(data[x].isna().sum())) for x in data.keys()]
display_as_tabs([(('{0} - {1} rows'.format(x, str(data[x].shape[0]))), summaries[idx]) for idx, x in enumerate(data.keys())])

Tabs
    [0] DataFrame(name='product_df - 1..., value=                          ...)
    [1] DataFrame(name='opportunity_df -..., value=                          ...)

## Merging

### Expected data validation rules

1. `Creation Date` is before `Decision Date`.
2. `transition_to_stage_name` values __["Stage 4", "Stage 9", "Stage 1", "Stage 8"]__ should have corresponding `opportunity_status` values __["Closed/Win", "Closed/Win", "Closed/Lost", "Closed/Lost"]__ respectively.
3. `Product Quantity` is an __integer__ value.
4. All Products should have `Product_Status` value other than __Closed__.
5. Dates Must be between 2015 to 2016

#### Great Expectations 

We will leverage the [Great Expectations](https://docs.greatexpectations.io/en/latest/) library for exploring the data quality & validating whether it conforms to expected business rules. 

#### Rules verification

In [17]:
verification_dict = {}
# Rule tests
test_success_flag = True
decison_days = ge.from_pandas(pd.DataFrame(data['opportunity_df']['decision date'] - data['opportunity_df']['creation date']))
# print(decison_days)
verification_dict['rule_1'] = decison_days.expect_column_values_to_be_between(0,min_value=0, max_value=730
                                                    ,result_format="BASIC", include_config=True).to_json_dict()
test_success_flag = verification_dict['rule_1']['success']
# Rule 1
if test_success_flag:
    print(('Rule {} passed').format(1))
else:
    print(('Rule {} failed').format(1))

Rule 1 failed


In [18]:
stage_4_9 = data['opportunity_df'][data['opportunity_df']["transition to stage"] == ("Stage 4" and "Stage 9")]['opportunity status'] != 'Closed/Won'
stage_1_8 = data['opportunity_df'][data['opportunity_df']["transition to stage"] == ("Stage 1" and "Stage 8")]['opportunity status'] != 'Closed/Lost'

verification_dict['rule_2_1'] = ge.from_pandas(pd.DataFrame(stage_4_9)).expect_multicolumn_sum_to_equal(['opportunity status'], sum_total=0).to_json_dict()
verification_dict['rule_2_2'] = ge.from_pandas(pd.DataFrame(stage_1_8)).expect_multicolumn_sum_to_equal(['opportunity status'], sum_total=0).to_json_dict()

test_success_flag_1 = verification_dict['rule_2_1']['success']
test_success_flag_2 = verification_dict['rule_2_2']['success']

# Rule 1
if test_success_flag_1:
    print(('Rule {} passed').format('2.1'))
else:
    print(('Rule {} failed').format('2.1'))
    
if test_success_flag_2:
    print(('Rule {} passed').format('2.2'))
else:
    print(('Rule {} failed').format('2.2'))

Rule 2.1 passed
Rule 2.2 failed


In [19]:
verification_dict['rule_3'] = ge.from_pandas(data['product_df'][['product quantity']]).expect_column_values_to_be_of_type('product quantity', type_='int64').to_json_dict()
test_success_flag = verification_dict['rule_3']['success']

if test_success_flag:
    print(('Rule {} passed').format('3'))
else:
    print(('Rule {} failed').format('3'))

Rule 3 passed


In [20]:
verification_dict['rule_4'] = ge.from_pandas(data['product_df'].groupby(['product id'])['product status'].apply(lambda x: len(list(set(x)-{'Pending'})))).expect_column_values_to_be_between('product status', min_value=1).to_json_dict()
test_success_flag = verification_dict['rule_4']['success']

if test_success_flag:
    print(('Rule {} passed').format('4'))
else:
    print(('Rule {} failed').format('4'))

Rule 4 failed


In [21]:
min_date = 42004
max_date = 42734

verification_dict['rule_5_1'] = ge.from_pandas(pd.DataFrame(data['opportunity_df'][['decision date']])).expect_column_values_to_be_between('decision date', min_value=min_date, max_value=max_date).to_json_dict()
verification_dict['rule_5_2'] = ge.from_pandas(pd.DataFrame(data['opportunity_df'][['creation date']])).expect_column_values_to_be_between('creation date', min_value=min_date, max_value=max_date).to_json_dict()

test_success_flag_1 = verification_dict['rule_5_1']['success']
test_success_flag_2 = verification_dict['rule_5_2']['success']

# Rule 1
if test_success_flag_1:
    print(('Rule {} passed').format('5.1'))
else:
    print(('Rule {} failed').format('5.1'))
    
if test_success_flag_2:
    print(('Rule {} passed').format('5.2'))
else:
    print(('Rule {} failed').format('5.2'))

Rule 5.1 failed
Rule 5.2 passed


### Fixes for failed rules

#### Fixing Rule 1

In [22]:
print('# Unexpected Values: {0}'.format(verification_dict['rule_1']['result']['unexpected_count']))

# Unexpected Values: 12380


* After looking at the data points which have decision_date before creation_date, they start with transition_from_stage being "final" stage, i.e., any of following, Stage 1, 4, 8, 9.
* That brings a possibility of opportunity being accepted previously but the data point being created later.

#### Fixing Rule 2.2

In [23]:
print('# Unexpected Values: {0}'.format(verification_dict['rule_2_2']['result']['unexpected_count']))

from statistics import mode
dummy_op = data['opportunity_df'].copy()
map_obj = dummy_op.groupby('transition to stage')['opportunity status'].apply(lambda x: mode(x)).to_dict()
dummy_op['opportunity status'] = dummy_op['transition to stage'].map(map_obj) 

# Test after fixing
stage_1_8 = dummy_op[dummy_op["transition to stage"] == ("Stage 1" and "Stage 8")]['opportunity status'] != 'Closed/Lost'
test_2_2 = ge.from_pandas(pd.DataFrame(stage_1_8)).expect_multicolumn_sum_to_equal(['opportunity status'], sum_total=0).to_json_dict()

if test_2_2['success']:
    print('Rule 2.2 passed')
else:
    print('Rule 2.2 failed')

# Unexpected Values: 2
Rule 2.2 passed


#### Fixing Rule 4

In [24]:
print('# Unexpected Values: {0}'.format(verification_dict['rule_4']['result']['unexpected_count']))

# Unexpected Values: 908


* Opportunities being left out in pending without being provided with the 'Closed/Won' or 'Closed/Lost' are hard to be processed.
* So, the cases other than 'Closed/Won' are considered as 'Lost'.
* Also the winning of product will decide the winning of opportunity, than the winning of opportunity itself.

#### Fixing Rule 5.1

In [25]:
print('# Unexpected Values: {0}'.format(verification_dict['rule_5_1']['result']['unexpected_count']))

data['opportunity_df']['decision date'].sort_values()
pd.to_datetime(115131, unit='D', origin='1899-12-30')

# Unexpected Values: 7679


Timestamp('2215-03-20 00:00:00')

* Decision date is used to know the `days difference` between creation and decision.
* Most of the values fall before `2015` are due to decision date being before creation date.
* 2 values have decision date as `115131` which translates to `2215-03-20` which is clearly a mistake with 200 years. This value should be replaced with `2015-03-20`.

## Health Analysis

Get an overview of the overall health of your dataset. This is usually quick to compute and hopefully highlights some problems to focus on.



### Summary Plot

Provides a high level summary of the health your dataset.

**Watch out for:**

* too few numeric values
* high % of missing values
* high % of duplicate values
* high % of duplicate columns 

In [26]:
summaries_and_plots = [analysis.get_data_health_summary(data[x], return_plot=True) for x in data.keys()]
plots = [x[1] for x in summaries_and_plots]
display_as_tabs([(x, plots[idx]) for idx, x in enumerate(data.keys())])

Tabs
    [0] HoloViews(Layout, name='product_df')
    [1] HoloViews(Layout, name='opportunity_df')

**Dev NOTES**

<details>
1. Datatypes : We have both numeric and other types. The bulk of them seem to be numeric. `Numeric` is defined to be one of [float|int|date] and the rest are categorized as `Others`. A column is assumed to have `date` values if it has the string `date` in the column name.

**[TODO]** We prob. need more types: integral, float, bool, dates/timestamps, strings. We have this functionality in Dataprocessor.

2. The missing value plot seems to indicate missing values are not present but we do have them. 

**[TODO]** The plot can be improved to better display small values

3. We are looking for duplicate observations (rows in the data). The plot shows the % of rows that are an exact replica of another row (using `df.duplicated`)

4. We are looking for duplicate features (columns in the data).

**[TODO]** The tigerml code seems complicated but it looks like we compare each column against all other similar columns (numeric/categoric) after dropping nans, infs


**[TODO]** We need better data inspectors. The current data inspectors show columns from the dataframe used to construct the plot and **not** the original data. This does not make sense for an end-user who didn't expicitly construct the intermediate data used for the plot. It would be more meaningful to have labels that match the legends (e.g unique_columns:100%, duplicate_columns:0). Also, the y-axis label doesen't tell anything. The x-axis prob. needs an axis (0 to 100%).

</details>

### Missing Values summary

This provides an overall view focussing on amount of missing values in the dataset.

**Watch out for:**
* A few columns have significant number of missing values 
* Most columns have significant number of missing values


In [27]:
summaries_and_plots = [analysis.get_missing_values_summary(data[x], return_plot=True) for x in data.keys()]
plots = [x[1] for x in summaries_and_plots]
display_as_tabs([(x, plots[idx]) for idx, x in enumerate(data.keys())])

Tabs
    [0] HoloViews(Bars, name='product_df')
    [1] HoloViews(Bars, name='opportunity_df')

**Dev notes:**

<details>
    
    * By default, the following are considered missing/NA values : `[np.Nan, pd.NaT, 'NA', None]`
    * additional values can be passed to tigerml (add_additional_na_values)
    * these are applied to all columns.
    
    * some of the above information can be learnt from the data discovery step (see discussion below)
    
</details>

### Duplicate Columns

In [28]:
summaries = [analysis.get_duplicate_columns(data[x]) for x in data.keys()]
display_as_tabs([(x, summaries[idx]) for idx, x in enumerate(data.keys())])

Tabs
    [0] Markdown(str, name='product_df')
    [1] Markdown(str, name='opportunity_df')

### Outlier Checks

In [29]:
summaries = [analysis.get_outliers(data[x]) for x in data.keys()]
display_as_tabs([(x, summaries[idx]) for idx, x in enumerate(data.keys())])

Tabs
    [0] DataFrame(name='product_df', value=                  ('Data S...)
    [1] DataFrame(name='opportunity_df', value=                          ...)

## Health Analysis report

Generate a report that has all the above data in a single html. This could be useful to submit to a client

In [32]:
os.environ['DEBUG'] = 'true'
health_reports_gen = [health.summary_report(data[x],save_path=r'./health_report_'+x+'.html') for x in data.keys()]